In [2]:
import pandas as pd
import os
import time
import requests
import json
import csv
from tqdm import tqdm

import tweepy

import requests
import pandas as pd
import os
import collections
from langdetect import detect
import matplotlib.pyplot as plt

f = open("api-keys.json")
input=json.load(f)

In [3]:
import nltk
import string 
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

# GET STOPWORDS
from nltk.corpus import stopwords
stop_words=nltk.corpus.stopwords.words('english')

# INITALIZE STEMMER+LEMITZIZER+SIA
sia = SentimentIntensityAnalyzer()
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [4]:
# LOAD KEYS INTO API
consumer_key=input["consumer_key"]    
consumer_secret=input["consumer_secret"]    
access_token=input["access_token"]    
access_token_secret=input["access_token_secret"]    
bearer_token=input["bearer_token"]    


In [5]:
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)

api = tweepy.API(auth)
headers = {"Authorization": "Bearer {}".format(bearer_token)}
print(api)
print(api.verify_credentials().screen_name)

monk_digi


In [6]:
def search_more(query,tweets_num):
    num_tweets_collected=0
    searches=[]
    while num_tweets_collected<tweets_num:
        if len(searches)==0:
            search_results = api.search_tweets(query,lang="en", count=100)
        else:
            search_results = api.search_tweets(query,lang="en", count=100,max_id=max_id_next)
        num_tweets_collected+=len(search_results)
        max_id_next=int(search_results[-1]._json["id_str"])-1

        for i in range(len(search_results)):
            searches.append(search_results[i])
    return(searches)

In [16]:
def sh_to_list(searches):
    texts=[]
    for i in range(len(searches)):
        texts.append(searches[i]._json["text"])
    return texts


In [8]:
#acquire data from twitter

sh_free=search_more("freeway",3000)
sh_free_list=list(set(sh_to_list(sh_free)))
sh_mtr=search_more("metro",3000)
sh_mtr_list=list(set(sh_to_list(sh_mtr)))

In [9]:
# clean the text data

def clean_string(text):
    #FILTER OUT UNWANTED CHAR
    new_text=""
    keep=" abcdefghijklmnopqrstuvwxyz0123456789"
    for character in text:
        if character.lower() in keep:
            new_text+=character.lower()
        else: 
            new_text+=" "
    text=new_text

    #FILTER OUT UNWANTED WORDS
    new_text=""
    for word in nltk.tokenize.word_tokenize(text):
        if word not in nltk.corpus.stopwords.words('english'):
            #lemmatize 
            tmp=lemmatizer.lemmatize(word)
            word=tmp
            if len(word)>1:
                if word in [".",",","!","?",":",";"]:
                    #remove the last space
                    new_text=new_text[0:-1]+word+" "
                else: #add a space
                    new_text+=word.lower()+" "
    text=new_text.strip()
    return text

In [10]:
for i in range(len(sh_free_list)):
    sh_free_list[i]=clean_string(sh_free_list[i])

In [11]:
for i in range(len(sh_mtr_list)):
    sh_mtr_list[i]=clean_string(sh_mtr_list[i])

In [12]:
tmp1=[]
for i in range(0,len(sh_free_list)):
    tmp1.append([sh_free_list[i],"freeway"])
df1=pd.DataFrame(tmp1)
df1=df1.rename(columns={0: "text", 1: "label"})
print(df1)

                                                   text    label
0     woman accused killing mother wednesday morning...  freeway
1     people think taking train poor people train ex...  freeway
2     natural history museum los angeles county 1975...  freeway
3     fwt frreway message freeway team telegram hope...  freeway
4                         ahhh drove freeway first time  freeway
...                                                 ...      ...
2497         nowplaying freeway tune http co qvgzclbgcu  freeway
2498  alejando816 modot kc imagine freeway downtown ...  freeway
2499  laverton north heavy traffic citybound prince ...  freeway
2500  joel krueger freeway adviser called global chi...  freeway
2501  someone say needed people believe oh never mon...  freeway

[2502 rows x 2 columns]


In [13]:
tmp2=[]
for i in range(0,len(sh_mtr_list)):
    tmp2.append([sh_mtr_list[i],"metro"])
df2=pd.DataFrame(tmp2)
df2=df2.rename(columns={0: "text", 1: "label"})
print(df2)

                                                   text  label
0     jaredpolis kyleclark contrast former republica...  metro
1     bluedemondegen following night saw rage machin...  metro
2     tom glynn carney got meet 102 year old sa hero...  metro
3     rt nicklima7 quarter rhode island top local el...  metro
4     brewenor afkaduddy charliehustleco part jayhaw...  metro
...                                                 ...    ...
1941              notoriousdmm luke metro sugondeez nut  metro
1942  wuhan back lockdown nearly three year first co...  metro
1943  hispanic population metro atlanta doubled sinc...  metro
1944  ct plan spend 12 billion improve train service...  metro
1945  rt susanbolger delighted launch halloweenestac...  metro

[1946 rows x 2 columns]


In [14]:
df=pd.concat([df1,df2])
print(df)
df.to_csv('free_mtr_text_data.csv',index=False)

                                                   text    label
0     woman accused killing mother wednesday morning...  freeway
1     people think taking train poor people train ex...  freeway
2     natural history museum los angeles county 1975...  freeway
3     fwt frreway message freeway team telegram hope...  freeway
4                         ahhh drove freeway first time  freeway
...                                                 ...      ...
1941              notoriousdmm luke metro sugondeez nut    metro
1942  wuhan back lockdown nearly three year first co...    metro
1943  hispanic population metro atlanta doubled sinc...    metro
1944  ct plan spend 12 billion improve train service...    metro
1945  rt susanbolger delighted launch halloweenestac...    metro

[4448 rows x 2 columns]
